<a href="https://colab.research.google.com/github/kg4-ken1ro/mypandas_tutorial/blob/main/study_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import warnings
import pandas as pd
import xgboost as xgb

#警告文無視
warnings.filterwarnings('ignore')

#driveのcsvファイルを読み込む
df_train = pd.read_csv('drive/My Drive/train.csv')
df_test = pd.read_csv("drive/My Drive/test.csv")

In [32]:
#前の5行(0から)を読み込む
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [33]:
#前の5行(0から)を読み込む
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [34]:
#欠損値(NaN)が含まれていないか確認
#Trueなら含まれている
print("--train--")
for col in df_train.columns:
  if df_train[col].isnull().any():
    print(col, df_train[col].isnull().any())

--train--
Age True
Cabin True
Embarked True


In [35]:
#欠損値(NaN)が含まれていないか確認
#Trueなら含まれている
print("==test==")
for col in df_test.columns:
  if df_test[col].isnull().any():
    print(col, df_test[col].isnull().any())

==test==
Age True
Fare True
Cabin True


In [36]:
#train前処理
#df["列名"] = df["列名"].fillna(値) で特定の列の欠損値を埋める
df_train["Age"] = df_train["Age"].fillna(df_train["Age"].mean())
df_train["Cabin"] = df_train["Cabin"].fillna(df_train["Cabin"].mode()[0])
df_train["Embarked"] = df_train["Embarked"].fillna(df_train["Embarked"].mode()[0])

In [37]:
#test前処理
#df["列名"] = df["列名"].fillna(値) で特定の列の欠損値を埋める
df_test["Age"] = df_test["Age"].fillna(df_test["Age"].mean())
df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].mean())
df_test["Cabin"] = df_test["Cabin"].fillna(df_test["Cabin"].mode()[0]) 

In [38]:
#再確認
print("--train--")
for col in df_train.columns:
  if df_train[col].isnull().any():
    print(col, df_train[col].isnull().any())

--train--


In [39]:
print("==test==")
for col in df_test.columns:
  if df_test[col].isnull().any():
    print(col, df_test[col].isnull().any())

==test==


In [40]:
#train前処理
#dropで列の削除
df_train.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
#値を置換
df_train["Sex"] = df_train["Sex"].replace({"male": 0, "female": 1})
#ダミー変数化
df_train = pd.get_dummies(df_train)

In [41]:
#test前処理
#dropで列の削除
df_test.drop(["Name", "Ticket", "Cabin"], axis=1, inplace=True)
#値を置換
df_test["Sex"] = df_test["Sex"].replace({"male": 0, "female": 1})
#ダミー変数化
df_test = pd.get_dummies(df_test)
#testにsurvived列はないのでdrop不要
test_x = df_test

In [42]:
#説明変数、目的変数へ分割
train_y = df_train["Survived"]
train_x = df_train.drop("Survived", axis=1)

In [43]:
#機械学習アルゴリズム宣言、学習
clf = xgb.XGBClassifier()
clf.fit(train_x, train_y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [44]:
#予測
result = clf.predict(test_x)
pd.DataFrame({
    "PassengerId": test_x["PassengerId"],
    "Survived": result
}).to_csv("result.csv", index=False)

In [45]:
#colabにてデーターダウンロードに必要
from google.colab import files
files.download("result.csv")
#download後、kaggleにてsubmit
#firstは0.77751でした

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>